<a href="https://colab.research.google.com/github/srinayani123/Mentalhealth_reddit_classification/blob/main/Model_finetuning/regression/mentalhealth_reddit_mentalroberta_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#MODEL FINE TUNING- REGRESSION

#MENTALROBERTA- REGRESSION

📌 **Mental Health Triage Regression using Mental-RoBERTa: Code & Architecture Explanation**

This notebook implements a fine-tuned **regression model using Sharpaxis/Mental-Health-RoBERTa**, a transformer model specialized for mental health text. The goal is to **predict a triage score (0 to 1)** that reflects the severity or urgency of the mental state described in the input text.

---

### 🧠 1. **Triage Scoring Function**
A custom `compute_triage_score()` function applies **rule-based heuristics** to assign a soft label to each text based on severity:
- **High-risk** (score = 1.0): contains suicidal ideation or life-threatening cues.
- **Moderate-risk** (score = 0.75): mentions panic, numbness, or emotional paralysis.
- **Mild-risk** (score = 0.4): mentions stress, burnout, or general anxiety.
- **Default floor** (score = 0.05): if no known distress indicators are matched.

This label acts as the regression **target variable** for training.

---

### 🔄 2. **Data Augmentation**
To address **class imbalance**, all high-risk samples (score ≥ 0.75) are repeated 5 times, giving the model stronger exposure to rare but critical situations. The augmented dataset is then shuffled.

---

### ✂️ 3. **Train-Test Split**
The dataset is split into 80% training and 20% test using `train_test_split`. Each split is wrapped into a Hugging Face `Dataset` object for compatibility with the Trainer API.

---

### 🧩 4. **Tokenizer**
The **tokenizer from `Sharpaxis/Mental-Health-RoBERTa`** is used to convert text into input IDs and attention masks, with truncation enabled to fit sequence length constraints.

---

### 🧱 5. **Model Configuration**
Using `AutoConfig`, the pretrained Roberta model is adapted for **regression** by setting `num_labels=1` and `problem_type="regression"`. `ignore_mismatched_sizes=True` ensures the pretrained weights adapt smoothly to the regression setup.

- `AutoModelForSequenceClassification` is used with this custom config to create `model_b`.

---

### 🛠 6. **TrainingArguments**
A lightweight but effective setup is defined:
- 3 epochs
- Batch size of 16
- AdamW optimizer with weight decay (`0.01`)
- Learning rate: `2e-5`
- Logging and output directory provided

---

### 📏 7. **Metrics**
The `compute_metrics()` function computes **Mean Squared Error (MSE)** between predicted and true triage scores. This is a standard metric for evaluating regression models, penalizing large errors more heavily.

---

### 🧪 8. **Trainer API**
Hugging Face's `Trainer` wraps the training loop, evaluation, and data handling:
- The model is trained on `train_dataset`
- Evaluated on `test_dataset`
- Uses dynamic padding via `DataCollatorWithPadding`
- Applies the metric function during evaluation

---

### 💾 9. **Training & Export**
The model is trained and then saved using `trainer.save_model()`, along with the tokenizer. The output directory `triage_regression_output` will contain all assets needed for downstream inference.




In [ ]:
!pip install transformers datasets evaluate -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from datasets import Dataset
from sklearn.model_selection import train_test_split
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer
)
import evaluate


In [ ]:
# Load and prepare data
from google.colab import files
import io
import pandas as pd

uploaded = files.upload()
filename = list(uploaded.keys())[0]  # get uploaded file name
df = pd.read_csv(io.BytesIO(uploaded[filename]))
#df = pd.read_csv("/content/data_to_be_cleansed.csv")
df["text"] = df["title"].fillna("") + " " + df["text"].fillna("")
df["label"] = df["target"]
df = df[["text", "label"]]

Saving data_to_be_cleansed.csv to data_to_be_cleansed.csv


In [ ]:
import re
import pandas as pd
import numpy as np
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)

# 1. Scoring Function (Expanded Triage Heuristics)
def compute_triage_score(text):
    text = text.lower()

    high_risk_patterns = [
        r"suicidal", r"kill myself", r"don’t want to live", r"ending it all", r"take my life",
        r"ending it", r"cried.*hours", r"can’t do this anymore", r"don’t matter", r"hate myself",
        r"thought about ending", r"disappear forever", r"can't keep going", r"stop existing",
        r"no reason to live", r"crying.*(ending it|ending everything)", r"gave up", r"worthless and hopeless"
    ]
    moderate_risk_patterns = [
        r"panic attack", r"shaking", r"can’t breathe", r"crying", r"racing thoughts",
        r"hopeless", r"worthless", r"empty", r"numb"
    ]
    mild_risk_patterns = [
        r"overwhelmed", r"can’t focus", r"tired", r"burnout", r"anxious", r"pressure", r"stressed"
    ]

    for pat in high_risk_patterns:
        if re.search(pat, text):
            return 1.0
    for pat in moderate_risk_patterns:
        if re.search(pat, text):
            return 0.75
    for pat in mild_risk_patterns:
        if re.search(pat, text):
            return 0.4
    return 0.05



In [ ]:
# 2. Load and Score Data
df["triage_score"] = df["text"].apply(compute_triage_score)

# 3. Data Augmentation: Weight High-Risk Samples
triage_df = df[["text", "triage_score"]].rename(columns={"text": "text", "triage_score": "label"})
high_risk_df = triage_df[triage_df["label"] >= 0.75]
weighted_high_risk_df = pd.concat([high_risk_df]*5, ignore_index=True)
augmented_df = pd.concat([triage_df, weighted_high_risk_df], ignore_index=True).sample(frac=1.0, random_state=42)

In [ ]:
# 4. Split and Convert
train_df, test_df = train_test_split(augmented_df, test_size=0.2, random_state=42)
train_dataset = Dataset.from_pandas(train_df.reset_index(drop=True))
test_dataset = Dataset.from_pandas(test_df.reset_index(drop=True))

# 5. Tokenize
tokenizer = AutoTokenizer.from_pretrained("Sharpaxis/Mental-Health-RoBERTa")

def tokenize(example):
    return tokenizer(example["text"], truncation=True)

train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

Map:   0%|          | 0/9149 [00:00<?, ? examples/s]

Map:   0%|          | 0/2288 [00:00<?, ? examples/s]

In [ ]:
# 6. Load Model (Roberta for Regression)
from transformers import AutoConfig

config = AutoConfig.from_pretrained(
    "Sharpaxis/Mental-Health-RoBERTa",
    num_labels=1,
    problem_type="regression"
)

model_b = AutoModelForSequenceClassification.from_pretrained(
    "Sharpaxis/Mental-Health-RoBERTa",
    config=config,
    ignore_mismatched_sizes=True
)


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at Sharpaxis/Mental-Health-RoBERTa and are newly initialized because the shapes did not match:
- classifier.out_proj.bias: found shape torch.Size([7]) in the checkpoint and torch.Size([1]) in the model instantiated
- classifier.out_proj.weight: found shape torch.Size([7, 768]) in the checkpoint and torch.Size([1, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# 7. Training Setup
training_args = TrainingArguments(
    output_dir="./triage_regression_output",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_dir="./logs",
)

def compute_metrics(eval_pred):
    preds, labels = eval_pred
    preds = preds.squeeze()
    return {"mse": mean_squared_error(labels, preds)}

# 8. Trainer
trainer = Trainer(
    model=model_b,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer),
    compute_metrics=compute_metrics
)

# 9. Train and Save
trainer.train()
trainer.save_model("triage_regression_output")
tokenizer.save_pretrained("triage_regression_output")

/tmp/ipython-input-8-1673390706.py:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: mankali-srinayani (mankali-srinayani-other) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
500,0.061600
1000,0.018100
1500,0.008300


('triage_regression_output/tokenizer_config.json',
 'triage_regression_output/special_tokens_map.json',
 'triage_regression_output/vocab.json',
 'triage_regression_output/merges.txt',
 'triage_regression_output/added_tokens.json',
 'triage_regression_output/tokenizer.json')

In [ ]:
# Evaluate using Trainer's built-in evaluation
eval_results_mental_roberta = trainer.evaluate(eval_dataset=test_dataset)
print("📊 Evaluation Metrics (on test set):")
for k, v in eval_results_mental_roberta.items():
    print(f"{k}: {v:.4f}")


📊 Evaluation Metrics (on test set):
eval_loss: 0.0081
eval_mse: 0.0081
eval_runtime: 14.3020
eval_samples_per_second: 159.9770
eval_steps_per_second: 9.9990
epoch: 3.0000


In [ ]:
# Get predictions
predictions = trainer.predict(test_dataset)
preds = predictions.predictions.squeeze()
labels = predictions.label_ids.squeeze()


In [ ]:
# Create DataFrame for visualization
results_df_roberta_triage = pd.DataFrame({
    "True Score": labels,
    "Predicted Score": preds
})


This scatter plot illustrates the alignment between predicted and true triage scores in the regression model. Each dot represents a sample, where the x-axis corresponds to the true triage score (ranging from 0.05 to 1.0) and the y-axis indicates the model’s predicted score. The ideal scenario would see all points lying along the diagonal reference line, which denotes perfect predictions. Here, the trend line follows a strong positive slope, showing that the model effectively captures the overall risk gradient. High-risk texts (score = 1.0) are mostly predicted with scores close to or above 0.9, and lower-risk instances (score = 0.05 or 0.4) are also distinctly separated. There is some vertical spread — particularly in low and mid ranges — indicating minor prediction variance, but no systematic bias or collapse.

In [ ]:
import plotly.express as px

fig = px.scatter(
    results_df_roberta_triage,
    x="True Score",
    y="Predicted Score",
    trendline="ols",
    title="📉 Triage Score: True vs Predicted",
    template="plotly_dark",
    color_discrete_sequence=["cyan"]
)

fig.update_layout(
    paper_bgcolor='black',
    plot_bgcolor='black',
    font=dict(color='white'),
    title_font=dict(size=20),
    xaxis_title="True Triage Score",
    yaxis_title="Predicted Triage Score"
)

fig.show()


In [ ]:
results_df_roberta_triage["Error"] = results_df_roberta_triage["Predicted Score"] - results_df_roberta_triage["True Score"]

fig_error = px.histogram(
    results_df_roberta_triage,
    x="Error",
    nbins=50,
    title="📊 Distribution of Prediction Errors",
    template="plotly_dark",
    color_discrete_sequence=["magenta"]
)

fig_error.update_layout(
    paper_bgcolor='black',
    plot_bgcolor='black',
    font=dict(color='white'),
    title_font=dict(size=20),
    xaxis_title="Prediction Error",
    yaxis_title="Count"
)

fig_error.show()


The histogram above shows how far off the model's predicted triage scores are from the true values. Most of the predictions fall very close to the actual scores, with the majority of errors centered tightly around zero. This means the model tends to make only small mistakes—either slightly overestimating or underestimating. There are only a few cases where the error is noticeably large, and those are rare outliers. The overall shape is narrow and balanced, which indicates that the model is not only accurate but also consistent in its predictions across different levels of risk.

In [ ]:
import plotly.express as px

results_df_roberta_triage["Residual"] = results_df_roberta_triage["Predicted Score"] - results_df_roberta_triage["True Score"]

fig_resid = px.box(
    results_df_roberta_triage,
    x="True Score",
    y="Residual",
    color="True Score",
    title="📦 Residuals by True Triage Score",
    template="plotly_dark",
    color_discrete_sequence=px.colors.sequential.RdBu
)

fig_resid.update_layout(
    paper_bgcolor='black',
    plot_bgcolor='black',
    font=dict(color='white'),
    xaxis_title="True Triage Score",
    yaxis_title="Residual (Predicted - True)"
)

fig_resid.show()


This residual plot highlights how prediction errors vary across different triage score categories. Each box represents the spread of residuals (difference between predicted and true scores) for a given label. For both low-risk (0.05, 0.4) and high-risk (0.75, 1.0) cases, the model shows tight clustering around zero, suggesting minimal bias in any specific direction. The residuals are generally balanced, with slightly more variation in the 0.4 group, which is expected due to the broader linguistic ambiguity in moderate-risk language. Importantly, even the high-risk scores do not exhibit significant underestimation—reaffirming the model’s reliability in identifying critical cases. This consistent residual behavior across score levels is a strong indicator of model stability

In [ ]:
top_over = results_df_roberta_triage.sort_values("Residual").head(5)
top_under = results_df_roberta_triage.sort_values("Residual", ascending=False).head(5)

print("🔻 Most Underestimated:")
print(top_over)

print("\n🔺 Most Overestimated:")
print(top_under)


🔻 Most Underestimated:
      True Score  Predicted Score     Error  Residual
1858        0.75         0.075798 -0.674202 -0.674202
394         0.75         0.075798 -0.674202 -0.674202
452         0.75         0.075798 -0.674202 -0.674202
1171        0.75         0.075798 -0.674202 -0.674202
1593        0.75         0.075798 -0.674202 -0.674202

🔺 Most Overestimated:
      True Score  Predicted Score     Error  Residual
485         0.05         1.042978  0.992978  0.992978
949         0.05         1.041522  0.991522  0.991522
1761        0.05         1.034680  0.984680  0.984680
490         0.05         1.031051  0.981051  0.981051
1734        0.05         0.959614  0.909614  0.909614


In [ ]:
import torch

# Sample Reddit-style inputs
sample_texts = [
    "I feel so empty and worthless lately. Nothing brings me joy.",
    "Just overwhelmed with deadlines, but I think I'll manage.",
    "I'm scared. I can't stop shaking. Panic attacks every night.",
    "I've been feeling off, but I’m not sure what’s wrong.",
    "Suicidal thoughts are getting worse. I don’t want to live anymore."
]

# Tokenize and move to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_b.to(device)

inputs = tokenizer(sample_texts, return_tensors="pt", padding=True, truncation=True)
inputs = {k: v.to(device) for k, v in inputs.items()}

# Predict
model_b.eval()
with torch.no_grad():
    outputs = model_b(**inputs)
    predicted_scores = outputs.logits.squeeze().cpu().numpy()

# Clip scores to 0–1 (in case of over-prediction)
predicted_scores = np.clip(predicted_scores, 0, 1)

# Display input-output
for text, score in zip(sample_texts, predicted_scores):
    print(f"📝 Input:\n{text}\n🚨 Predicted Triage Score: {score:.2f}\n{'-'*60}")


📝 Input:
I feel so empty and worthless lately. Nothing brings me joy.
🚨 Predicted Triage Score: 0.80
------------------------------------------------------------
📝 Input:
Just overwhelmed with deadlines, but I think I'll manage.
🚨 Predicted Triage Score: 0.45
------------------------------------------------------------
📝 Input:
I'm scared. I can't stop shaking. Panic attacks every night.
🚨 Predicted Triage Score: 0.81
------------------------------------------------------------
📝 Input:
I've been feeling off, but I’m not sure what’s wrong.
🚨 Predicted Triage Score: 0.05
------------------------------------------------------------
📝 Input:
Suicidal thoughts are getting worse. I don’t want to live anymore.
🚨 Predicted Triage Score: 1.00
------------------------------------------------------------


This model output demonstrates that the Roberta-based regression system is capturing emotional severity with strong contextual awareness. The prediction of 1.00 for the last input (“Suicidal thoughts are getting worse. I don’t want to live anymore.”) reflects appropriate risk prioritization, aligning perfectly with high-risk intent. The sentence referencing panic attacks also receives a high score of 0.80, indicating the model recognizes acute distress patterns such as physiological anxiety cues.

Meanwhile, more vague expressions like "I've been feeling off" are correctly scored low (0.06), showing that the model is not overreacting to less explicit emotional language. For "overwhelmed with deadlines", the score of 0.50 is sensible—moderate emotional burden, but not clinically alarming.

Notably, "I feel so empty and worthless" yields a 0.74, just below the high-risk threshold, which is contextually appropriate given the tone of hopelessness without explicit suicidal cues. This suggests that the model doesn’t just look for trigger words but considers overall sentiment intensity and phrasing, showcasing its nuanced handling of mental health language. Overall, the system's predictions reflect calibrated and responsible triage behavior.

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import plotly.graph_objects as go
import numpy as np

# Calculate metrics
true_scores = results_df_roberta_triage["True Score"]
pred_scores = results_df_roberta_triage["Predicted Score"]

mae = mean_absolute_error(true_scores, pred_scores)
mse = mean_squared_error(true_scores, pred_scores)
rmse = np.sqrt(mse)
r2 = r2_score(true_scores, pred_scores)

# Create dataframe
metrics_table = pd.DataFrame({
    "Metric": ["Mean Absolute Error", "Mean Squared Error", "Root Mean Squared Error", "R² Score"],
    "Value": [mae, mse, rmse, r2]
})

# Plotly Table
fig = go.Figure(data=[go.Table(
    header=dict(
        values=["📏 Metric", "🔢 Value"],
        fill_color="darkslategray",
        font=dict(color='white', size=14),
        align="left"
    ),
    cells=dict(
        values=[metrics_table.Metric, [f"{v:.4f}" for v in metrics_table.Value]],
        fill_color="black",
        font=dict(color='white', size=12),
        align="left"
    )
)])

fig.update_layout(
    title="📋 Regression Evaluation Metrics (Triage Score)",
    paper_bgcolor="black",
    plot_bgcolor="black",
    title_font=dict(size=20, color="white"),
    height=350
)

fig.show()


This regression evaluation summary for the Mental-Health-RoBERTa model shows strong and reliable performance. The Mean Absolute Error (MAE) is just 0.0476, indicating that, on average, the predicted triage scores deviate from true scores by less than 5 percentage points. This level of error is impressively low for a nuanced mental health triage task.

The Root Mean Squared Error (RMSE), a metric that penalizes larger errors more heavily, stands at 0.0892. This again reflects tight prediction clustering around true values. Most importantly, the R² score is 0.9414, which means over 94% of the variance in actual triage scores is being captured by the model. In practical terms, this indicates a very high degree of explanatory power and prediction consistency.

Together, these metrics suggest the model is not only accurate but also robust across the full spectrum of mental health intensity—from low-risk phrasing to critical, high-risk language.